In [1]:
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
import numpy as np

### Part 1: Prediction quality vs feature selection

#### Tasks

In this taks you are supposed to

1. repeatedly simulate new datasets and for each dataset do Steps 2 and 3 
2.  determine λ_{min} and λ_{1se} using cross-validation on the training data

3.  compare the two resulting Lasso models with respect to
    - Mean squared error on the training as well as test data
    - Feature selection quality in comparison to the original simulated regression coefficients


Let us look a bit closer at each task.

1. Simulate data
Simulating useful data for the Lasso isn't complicated but to make sure you can focus on the interesting tasks I included a Python and a R version of a data-generating function. You can either ignore them and follow the description here, use them or take them as inspiration to write your own. The ideas are

    1. Generate n observations from an isometric normal distribution $N(0,Ip)$, i.e. theoretically uncorrelated features.
    2. Generate regression coefficients beta as a vector that contains ceiling((1 - sparsity) p) non-zero elements [ceiling = closest integer less or equal to] that are normal distributed with a standard deviation of beta_scale. All other elements are zero.
    3. At this point Xβ is the noise-less response and √∥Xβ∥22/(n−1) is its sample standard deviation. Signal-to-noise ratio is a measure for the proportion of the signal (noise-less response) standard deviation to the standard deviation of the noise, i.e. SNR = sd_signal / sd_noise. Specifying the SNR is a convenient way to determine what standard deviation for the noise is reasonable by choosing sd_noise = sd_signal / SNR. As an example, SNR = 2 means that the standard deviation of the noise-less is twice as large as the standard deviation of the noise and the noise-less response will be more "pronounced" the larger the SNR is. If 0 <= SNR < 1, then the noise is stronger than the signal which is hard to deal with for most methods.
    4. Finally, the response is created in the form of a linear model y = X beta + sigma eps.

In [2]:
def simulate_data(n, p, rng, *, sparsity=0.95, SNR=2.0, beta_scale=5.0):
    """Simulate data for Project 3, Part 1.

    Parameters
    ----------
    n : int
        Number of samples
    p : int
        Number of features
    rng : numpy.random.Generator
        Random number generator (e.g. from `numpy.random.default_rng`)
    sparsity : float in (0, 1)
        Percentage of zero elements in simulated regression coefficients
    SNR : positive float
        Signal-to-noise ratio (see explanation above)
    beta_scale : float
        Scaling for the coefficient to make sure they are large

    Returns
    -------
    X : `n x p` numpy.array
        Matrix of features
    y : `n` numpy.array
        Vector of responses
    beta : `p` numpy.array
        Vector of regression coefficients
    """
    X = rng.standard_normal(size=(n, p))
    
    q = int(np.ceil((1.0 - sparsity) * p))
    beta = np.zeros((p,), dtype=float)
    beta[:q] = beta_scale * rng.standard_normal(size=(q,))
    
    sigma = np.sqrt(np.sum(np.square(X @ beta)) / (n - 1)) / SNR

    y = X @ beta + sigma * rng.standard_normal(size=(n,))

    # Shuffle columns so that non-zero features appear
    # not simply in the first (1 - sparsity) * p columns
    idx_col = rng.permutation(p)
    
    return X[:, idx_col], y, beta[idx_col]

In [3]:
p = 500     # Fix p at something large, e.g. 500 or 1000
n_list = [125, 250, 375]    # Let n vary compared to p, e.g. iterate through [200, 500, 750] if you set p = 1000. What truly matters here is the ratio p / n, so if you choose p differently, adjust your choices for n
sparsities = [0.75, 0.9, 0.95, 0.99]    # Let sparsity vary for a few choices, e.g. [0.75, 0.9, 0.95, 0.99]
SNR = 2     # You can fix SNR at something reasonable like 2 or 5 throughout
beta_scale = 5      # Same holds for beta_scale, maybe 5 or 10
rng = np.random.default_rng(12345)

In addition, it will help you tremendously in the interpretation of the results if you **repeat the simulations a few times**, say 5 or 10 times, for each choice of n and sparsity. Here is why you should be careful with your choices: If you chose three values for n and four for sparsity, as well as 5 repeats, then you need to run your simulations for 60 datasets. A setup with 50 datasets took about 2 minutes on a 2017 MacBook, so if it takes hours, you did something wrong :-)

It can be good to include intermediate print-outs/clock output throughout the code, e.g. for iteration numbers or cross-validation so you can detect if there is a time sink somewhere

In [4]:
x, y, beta = simulate_data(n_list[0], p, rng)

Fit Lasso model

In [5]:
reg = LassoCV(cv=5, random_state=0).fit(x, y)
alpha = reg.alpha_
reg.score(x, y)

0.828751249920303

In [6]:
reg_2 = Lasso(alpha=alpha).fit(x,y)
reg_2.score(x, y)

0.828751249920303

In [7]:
reg.mse_path_.shape

(100, 5)

2. Determine hyperparameters

This works as described above the "Tasks" section. Depending on the the package you are using you will have to perform some different steps to get the coefficients and the predictions.

In [8]:
for n in n_list:
    for sparsity in sparsities:
        for i in range(5):
            x, y, beta = simulate_data(n, p, rng, sparsity=sparsity)
            lasso = LassoCV(cv=5).fit(x, y)
            mse_mat = lasso.mse_path_
            n_folds = mse_mat.shape[0]
            cv_mean = np.mean(mse_mat, axis=1)
            cv_std = np.std(mse_mat, axis=1)
            idx_min_mean = np.argmin(cv_mean)
            idx_alpha = np.where(
                (cv_mean <= cv_mean[idx_min_mean] + cv_std[idx_min_mean] / np.sqrt(n_folds)) &
                (cv_mean >= cv_mean[idx_min_mean])
            )[0][0]
            alpha_1se = lasso.alphas_[idx_alpha]
            print(alpha_1se)

10.752793759040847
18.41228025572191
19.87550066554975
17.84007855660928
8.451319633325255
2.309117994413889
4.168316154069316
5.7573150863919
1.9674058324493704
4.686999341335015
3.050204070587627
3.1091374245480536
3.2033887988794465
3.1246407717523303
1.4556675590140313
1.3383695086334695
1.2779497688572923
0.9634235300674979
1.6688346127258922
0.9667837279390837
2.6504013999173033
1.9798582660277964
2.812765470480938
2.5879015713764555
4.239692423135225
2.3214343541863203
1.9621086857561538
2.0447277846167418
1.413529001738004
1.908024730129443
0.9216855160251751
1.6288585461686438
1.2640874836169973
1.5483647807962941
1.0351721931081137
0.851909638110247
0.6848998542167665
0.5241395457717647
0.8683217402408382
0.9408574588691055
1.394420401052372
1.6425937379641709
1.6967259321789236
1.2134049764357242
1.966211360111008
1.2840264382529107
1.985171685158749
1.8387763880079742
1.2877046270520742
0.931159324422454
0.9015365884566127
1.2101449474325652
1.59729162187304
1.0547407587872

In [9]:
reg.mse_path_.shape[1]

5

### 3. Comparing the two Lasso models

#### Computation of train/test MSE

Whenever you use LassoCV or cv.glmnet you can, at the best, access the test error for each fold. To compute both the training and test MSE of the model, you have two options:

1. Write your own cross validation loop. Then you can evaluate the training and test error and safe both.
2. Instead of only creating a training dataset with the options given in Part 1, Task 1 you can do the following:
    - Fix n_test at some larger value, say, n_test = 500 or n_test = 1000
    - Instead of simulating n samples you generate now n + n_test and split the dataset. Train on the n samples with cross validation and use the remaining n_test samples for testing. You do not need to adapt the size of the test set to the size of the training dataset as long as you choose it large enough.
Option 2 is probably less tedious to implement, but the choice is up to you.

When reporting the results for the train/test MSE, **please do not simply report a table of numbers. Find a nice way to visualise the results.**

#### Question
 - How does the MSE of the $λ_{min}$ and $λ_{1se}$ models behave for different n and sparsity levels?

### Evaluating the feature selection capability

Compute sensitivity and specificity using the 0-1 codings of the true and estimated coefficient vectors and plot them in a scatter plot.


#### Questions

 - What differences between sensitivity/specificity computed from the $λ_{min}$ and the $λ_{1se}$ models can you observe?
 - How do different choices for n and sparsity affect the relationship of sensitivity/specificity?

### Part 2 - Selecting features with confidence

In [12]:
from sklearn.feature_selection import SelectKBest
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
import sys

if sys.platform == 'darwin':
    data = pd.read_csv('TCGA-PANCAN-HiSeq-801x20531/data.csv', index_col=0)
    labels = pd.read_csv('TCGA-PANCAN-HiSeq-801x20531/labels.csv', index_col=0)
else:
    data = pd.read_csv('TCGA-PANCAN-HiSeq-801x20531\data.csv', index_col=0)
    labels = pd.read_csv('TCGA-PANCAN-HiSeq-801x20531\labels.csv', index_col=0)

In [217]:
labels = (np.asarray(labels)).reshape(len(labels,))
k_features = 200
data_filtered = SelectKBest(k=k_features).fit_transform(data, labels)

/Users/Nisse/Library/Python/3.8/lib/python/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [    5    23  4370  4808  4809  4814  4816  4817  4831  5288  7661  7662
  7663  7664  7665  8121  9304  9306  9314  9316  9320  9452 10121 11958
 13991 14158 14159 14161 15138 15140 15141 15446 16566 16568 16569 16571
 16575 16578 16579 16604 16634 16637 16677 16697 16698 16699 16700 16701
 16702 16704 16705 16706 16707 16708 16709 16710 16711 16712 16713 16714
 16715 16716 16717 16718 16719 16720 16721 16722 16723 16724 16725 16726
 16727 16728 16729 16730 16731 16732 16733 16734 16735 16736 16737 16738
 16739 16740 16741 16742 16743 16744 16745 16746 16748 16749 16750 16751
 16752 16753 16754 16756 16757 16758 16759 16760 16761 16762 16763 16764
 16765 16766 16767 16768 16769 16770 16771 16772 16774 16775 16776 16777
 16778 16779 16780 16781 16782 16783 16785 16787 16788 16789 16790 16791
 16792 16794 16795 16796 16798 16799 16800 16801 16802 16803 1

Performe feature selection

In [77]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
hp = np.logspace(-4, 4, 30)
logresCV = LogisticRegressionCV(Cs=hp, cv=5, multi_class='ovr', solver='liblinear', intercept_scaling=1000, scoring='f1').fit(data_filtered, labels)
# logresCV.predict(data_filtered)
scoresCV = np.argmax(logresCV.scores_)
C_max = np.max(logresCV.C_)

In [214]:
# Cmax
C_max = logresCV.C_
# C_1se
argmax_over_classes = np.zeros(5)
sub_std = np.zeros(5)
mean_over_folds = np.zeros([5, 30])
std_over_classes = np.zeros([5, 30])
min_mean_over_classes = np.zeros(5)
where_min_mean = np.zeros(5)
for idx, key in enumerate(logresCV.scores_.keys()):
    current_class_score = logresCV.scores_[key]
    mean_over_folds[idx,:] = np.mean(current_class_score, axis=0)
    argmax_over_classes[idx] = np.argmax(mean_over_folds[idx,:])
    std_over_classes[idx,:] = np.std(current_class_score, axis=0)
    sub_std[idx] = mean_over_folds[idx, int(argmax_over_classes[idx])] - std_over_classes[idx,int(argmax_over_classes[idx])]
    min_mean_over_classes[idx] = np.min(mean_over_folds[idx,:][mean_over_folds[idx,:] >= sub_std[idx]])
    where_mean = np.where(mean_over_folds[idx,:] >= sub_std[idx])
#     and (mean_over_folds[cl,:] <= np.ones(30)*max_over_classes[cl]
#                idx_alpha = np.where(
#                (cv_mean <= cv_mean[idx_min_mean] + cv_std[idx_min_mean] / np.sqrt(n_folds)) &
#                (cv_mean >= cv_mean[idx_min_mean])
#            )[0][0]
    where_min_mean[idx] = where_mean[0][int(np.argmin(mean_over_folds[idx, where_mean]))]
print('STD for each class:', std_over_classes)
print('Max score over classes:', argmax_over_classes)
print('Lower bound for one standard deviation from maximum score:', sub_std)
print('Minimum mean score over classes:', min_mean_over_classes)
print('Minimum mean position over classes:', where_min_mean)
C_1se = hp[where_min_mean.astype(int)]
print('C_1se:', C_1se)
print('C_max:', C_max)

STD for each class: [[0.00404874 0.00404874 0.00404874 0.00404874 0.00404874 0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.01618685 0.01580316 0.01580316 0.01290323 0.01290323 0.01290323
  0.01290323 0.01290323 0.01290323 0.01290323 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.01419927 0.01428571 0.01428571 0.00701754 0.00701754 0.00701754
  0.00701754 0.00701754 0.00701754 0.00701754 0.00701754 0.00701754
  0.00701754 0.00701754 0.00701754 0.00701754 0.00701754 0.00701754
  0.00701754 0.00701754 0.00701754 0.00701754 0.00701754 0.00701754
  0.00701754 0.00701754 0.00701754 0.00701754 0.00701754 0.00701754]
 [0.02994365 

Bootstrapping

In [239]:
table_of_counts = np.zeros([5, k_features])
M = 100
for m in range(M):
    indicies = np.random.choice(len(labels), len(labels), replace=True)
    bootstrap_data = data_filtered[indicies, :]
    bootstrap_labels = labels[indicies]
    logres = LogisticRegression(C=np.max(C_max), penalty='l1', multi_class='ovr', solver='liblinear', random_state=0, intercept_scaling=1000).fit(bootstrap_data, bootstrap_labels)
    logres.predict(bootstrap_data)
    scores = logres.score(bootstrap_data, bootstrap_labels)
    coefficients = logres.coef_
    for genetype in range(5):
        features = np.where(coefficients[genetype, :] != 0)
        table_of_counts[genetype, features[0]] += 1

In [242]:
for m in range(M):
    for gene in range(5):
        np.where(table_of_counts[gene] )

array([  0.,   0.,   0.,   0.,   0.,  20.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,  65.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,  76.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,  31.,   0.,   0.,   0.,   0., 100.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 100.,  12.,
         0.,   0.,   0.,   0.,  32.,   0.,   0.,   0.,   0.,   0.,  99.,
         0.,   0.,   0.,  97.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,  64.,   0.,
         0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,  2

In [220]:
#np.shape(data_filtered)
indicies = np.random.choice(len(labels), len(labels), replace=True)
bootstrap_data = data_filtered[indicies, :]
bootstrap_labels = labels[indicies]

In [223]:
bootstrap_data

array([[ 0.        ,  0.        ,  9.70084889, ...,  3.57740466,
         2.69347581,  4.7897136 ],
       [ 0.        ,  0.        ,  9.50103701, ...,  0.53923327,
         0.53923327,  0.53923327],
       [10.15113073,  8.16893782, 12.45731162, ..., 11.19681725,
         9.88208512, 11.68032875],
       ...,
       [ 0.44911232,  0.        ,  8.52509792, ...,  1.29918603,
         1.06743221,  2.21794413],
       [ 1.11196609,  0.        ,  7.76687339, ...,  2.28436607,
         2.03463832,  2.03463832],
       [ 0.        ,  0.        ,  8.60620161, ...,  0.        ,
         2.83669169,  0.49528583]])

In [161]:
# counts = np.bincount(k)
values, count = np.unique(k, return_counts=True)
print(values[np.argmax(count)], np.max(count))

390 5
